In [1]:
import torch
from torch import nn

import torchvision
from torchvision.datasets import ImageFolder

from torchvision import transforms

from torch.utils.data import DataLoader
from pathlib import Path
from torchvision.models import resnet101

In [2]:
import sys
sys.path.append("..")

In [3]:
from video_classification.datasets import FolderOfFrameFoldersDataset, FrameWindowDataset

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
ROOT = Path("/home/ubuntu/SupervisedVideoClassification")
DATA_ROOT = Path(ROOT/"data")

In [6]:
train_transforms = transforms.Compose([
    torchvision.transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(p=0.25),
    transforms.RandomVerticalFlip(p=0.25),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

valid_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

In [7]:
train_ds = FolderOfFrameFoldersDataset(DATA_ROOT/'train', 
                                       transform=train_transforms, 
                                       base_class=FrameWindowDataset,
                                       window_size=3,
                                       overlapping=True,)
valid_ds = FolderOfFrameFoldersDataset(DATA_ROOT/'validation', 
                                       transform=valid_transforms, 
                                       base_class=FrameWindowDataset,
                                       window_size=3,
                                       overlapping=True,)

In [8]:
from torch import nn
from torchvision.models import resnet101
from video_classification.models.mlp import MLP


class SingleImageResNetModel(nn.Module):
    def __init__(self, mlp_sizes=[768, 128, 2]):
        super().__init__()
        resnet = resnet101(pretrained=True)
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)

        self.clf = MLP(2048, mlp_sizes)
        self.freeze_resnet()

    def forward(self, x):
        x = self.resnet(x).squeeze()
        x = self.clf(x)
        return x

    def freeze_resnet(self):
        for p in self.resnet.parameters():
            p.requires_grad = False

    def unfreeze_resnet(self):
        for p in self.resnet.parameters():
            p.requires_grad = True


In [9]:
from torch import nn


class AverageImagesModel(nn.Module):
    def __init__(self, mlp_sizes=[768, 128, 2]):
        super().__init__()
        self.single_image_model = SingleImageResNetModel(mlp_sizes)

    def forward(self, x):
        # x is of size (B, T, C, H, W)
        x = x.mean(1)  # We average all images in axis T
        x = self.single_image_model(x)  # and then it's business as usual
        return x

    def freeze_vgg(self):
        # Freeze the VGG classifier
        self.single_image_model.freeze_vgg()

    def unfreeze_vgg(self):
        self.single_image_model.unfreeze_vgg()


In [10]:
model = AverageImagesModel(
                 mlp_sizes=[1024, 256, 2]
)

model = model.to(device)

In [11]:
x = torch.stack([train_ds[0][0], train_ds[1][0], train_ds[2][0], train_ds[3][0]]).to(device)

In [12]:
model(x)

tensor([[-0.0064,  0.2145],
        [ 0.3605,  0.3722],
        [-0.1629,  0.2921],
        [-0.4763,  0.1611]], device='cuda:0', grad_fn=<AddmmBackward>)

In [13]:
from video_classification.trainer import Trainer

classes_weights = torch.Tensor([0.3, 1.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=classes_weights)

In [14]:
trainer = Trainer(train_ds, 
                  valid_ds, 
                  model, 
                  criterion,
                  "multi_frame_resnet101_from_scratch",
                  str(ROOT/'checkpoints'),
                  device=device,
                  amp_opt_level="O1",
                  cycle_mult=0.9,
                 )

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [15]:
trainer.train(lr=1e-3, 
              batch_size=64, 
              n_epochs=20,
              gradient_accumulation_steps=4,
              num_workers=8,
              max_gradient_norm=2.0,
             )

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


Training Results - Epoch: 1: Avg accuracy: 0.91 |Precision: 0.96, 0.42 |Recall: 0.94, 0.56 | F1: 0.72 | Avg loss: 0.38
Validation Results - Epoch: 1: Avg accuracy: 0.73 |Precision: 0.93, 0.15 |Recall: 0.77, 0.41 | F1: 0.53 | Avg loss: 0.55


Training Results - Epoch: 2: Avg accuracy: 0.86 |Precision: 0.97, 0.30 |Recall: 0.87, 0.71 | F1: 0.67 | Avg loss: 0.38
Validation Results - Epoch: 2: Avg accuracy: 0.52 |Precision: 0.95, 0.12 |Recall: 0.50, 0.73 | F1: 0.43 | Avg loss: 0.78


Training Results - Epoch: 3: Avg accuracy: 0.95 |Precision: 0.96, 0.82 |Recall: 0.99, 0.42 | F1: 0.77 | Avg loss: 0.29
Validation Results - Epoch: 3: Avg accuracy: 0.93 |Precision: 0.93, 0.79 |Recall: 0.99, 0.27 | F1: 0.69 | Avg loss: 0.43


Training Results - Epoch: 4: Avg accuracy: 0.94 |Precision: 0.96, 0.59 |Recall: 0.97, 0.51 | F1: 0.76 | Avg loss: 0.29
Validation Results - Epoch: 4: Avg accuracy: 0.87 |Precision: 0.93, 0.28 |Recall: 0.92, 0.32 | F1: 0.61 | Avg loss: 0.50


...

Training Results - Epoch: 16: Avg accuracy: 0.95 |Precision: 0.98, 0.63 |Recall: 0.97, 0.69 | F1: 0.81 | Avg loss: 0.21
Validation Results - Epoch: 16: Avg accuracy: 0.89 |Precision: 0.94, 0.37 |Recall: 0.95, 0.33 | F1: 0.64 | Avg loss: 0.52


Training Results - Epoch: 17: Avg accuracy: 0.96 |Precision: 0.97, 0.74 |Recall: 0.98, 0.65 | F1: 0.84 | Avg loss: 0.20
Validation Results - Epoch: 17: Avg accuracy: 0.92 |Precision: 0.93, 0.56 |Recall: 0.98, 0.30 | F1: 0.67 | Avg loss: 0.51


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0
Training Results - Epoch: 18: Avg accuracy: 0.96 |Precision: 0.97, 0.71 |Recall: 0.98, 0.68 | F1: 0.84 | Avg loss: 0.20
Validation Results - Epoch: 18: Avg accuracy: 0.92 |Precision: 0.94, 0.62 |Recall: 0.98, 0.30 | F1: 0.68 | Avg loss: 0.49


Training Results - Epoch: 19: Avg accuracy: 0.96 |Precision: 0.97, 0.73 |Recall: 0.98, 0.68 | F1: 0.84 | Avg loss: 0.20
Validation Results - Epoch: 19: Avg accuracy: 0.91 |Precision: 0.94, 0.51 |Recall: 0.97, 0.33 | F1: 0.68 | Avg loss: 0.45


Training Results - Epoch: 20: Avg accuracy: 0.95 |Precision: 0.97, 0.70 |Recall: 0.98, 0.67 | F1: 0.83 | Avg loss: 0.20
Validation Results - Epoch: 20: Avg accuracy: 0.91 |Precision: 0.93, 0.51 |Recall: 0.97, 0.30 | F1: 0.67 | Avg loss: 0.56


In [16]:
import pandas as pd
reform = {(outerKey, innerKey): values for outerKey, innerDict in trainer.epoch_state.items() for innerKey, values in innerDict.items()}
df = pd.DataFrame(reform).T
df

accuracy        f1       nll  \
1  train  0.910991  0.716367   0.37695   
   test   0.734655  0.528013  0.549095   
2  train  0.857893  0.672003  0.379128   
   test   0.521831  0.434401  0.778723   
3  train  0.950805  0.766638  0.289522   
   test   0.929551  0.685175  0.430452   
4  train  0.937709  0.757485  0.288317   
   test   0.868171  0.614446  0.502802   
5  train  0.942212  0.781067  0.261921   
   test   0.878296  0.629911  0.469749   
6  train  0.950317  0.802994  0.253573   
   test   0.926809  0.683918  0.447391   
7  train  0.938459  0.784201  0.248277   
   test   0.909935  0.657587  0.537911   
8  train  0.933506  0.779697  0.245756   
   test   0.836954  0.596454  0.645272   
9  train  0.946377  0.805765  0.235605   
   test   0.912677  0.672958  0.415801   
10 train  0.942287  0.800721  0.233027   
   test   0.895381  0.639172   0.51999   
11 train  0.941461  0.800052  0.230357   
   test   0.916051  0.680402  0.427468   
12 train  0.956208  0.823582  0.222684   
   test    0.91563  0.676114  0.457568   
13 train   0.95482  0.817077  0.232036   
   test   0.924278  0.672987   0.57181   
14 train  0.952268  0.828414  0.211018   
   test    0.91352  0.668939  0.517782   
15 train  0.950129  0.819358  0.211747   
   test   0.875976  0.620058  0.578824   
16 train   0.94679  0.813909  0.212972   
   test   0.891162  0.643735  0.516924   
17 train  0.957747  0.836159  0.203194   
   test   0.917317  0.672532  0.514596   
18 train  0.956058   0.83549  0.201526   
   test   0.921746  0.681308  0.487482   
19 train  0.957972  0.841164  0.202719   
   test   0.912466  0.676107  0.446676   
20 train  0.954933  0.831922  0.204358   
   test   0.913099  0.665466  0.561372   

                                          precision  \
1  train   [0.9641000041599068, 0.4218390804597701]   
   test    [0.9308211473565804, 0.1459915611814346]   
2  train  [0.9743938981204031, 0.30283365779796667]   
   test    [0.9500657030223391, 0.1240846216436127]   
3  train   [0.9558467899212107, 0.8229475766567754]   
   test    [0.9338555265448216, 0.7931034482758621]   
4  train    [0.961663391747203, 0.5887850467289719]   
   test   [0.9334114888628371, 0.28361344537815125]   
5  train   [0.9661255890764088, 0.6163556531284303]   
   test     [0.9351615152219381, 0.319634703196347]   
6  train   [0.9670518613299213, 0.6951219512195121]   
   test    [0.9344262295081968, 0.7168674698795181]   
7  train    [0.9704021228822208, 0.575208913649025]   
   test   [0.9338382713299176, 0.48412698412698413]   
8  train   [0.9728034336180925, 0.5397022332506204]   
   test    [0.9350330963471439, 0.2326283987915408]   
9  train    [0.971953892361393, 0.6330183988065639]   
   test    [0.9363555158552925, 0.5095057034220533]   
10 train    [0.9738382804503583, 0.595773381294964]   
   test     [0.9333937471681015, 0.382262996941896]   
11 train    [0.9743589743589743, 0.588830255057168]   
   test    [0.9369710467706014, 0.5418326693227091]   
12 train   [0.9690466693258876, 0.7523749208359721]   
   test    [0.9361654804270463, 0.5387755102040817]   
13 train   [0.9679540834628722, 0.7435897435897436]   
   test    [0.9331147540983606, 0.6807228915662651]   
14 train   [0.9757772810404389, 0.6692759295499021]   
   test    [0.9352469959946595, 0.5182186234817814]   
15 train    [0.973983277863463, 0.6578816509199403]   
   test    [0.9331786542923434, 0.3039443155452436]   
16 train   [0.9751664420209941, 0.6260387811634349]   
   test    [0.9354691075514874, 0.3692722371967655]   
17 train   [0.9726736487299867, 0.7424768518518519]   
   test    [0.9349269588313412, 0.5605381165919282]   
18 train    [0.9744935023004277, 0.711918760021379]   
   test    [0.9354198809786203, 0.6176470588235294]   
19 train    [0.9747753556030141, 0.730349344978166]   
   test    [0.9371223987469233, 0.5073529411764706]   
20 train   [0.9741943300218076, 0.7021727609962904]   
   test      [0.9346376167185416, 0.51440329218107]   

                               